<font size = 10>

# pimm依赖

## ***data系列***

<font size = 3>

### pimm.data.Dataset 与 pimm.data.create_loader

#### - import测试

In [1]:
from Paddle_Cream.lib.utils.pimm.data import Dataset, create_loader

C:\Users\LibrarristShalinward\AppData\Local\Programs\Python\Python38\lib\site-packages\paddle\fluid\framework.py:300: UserWarning: You are using GPU version Paddle, but your CUDA device is not set properly. CPU device will be used by default.
  warnings.warn(


#### - 测试主要方法与函数运行

In [2]:
dataset_eval = Dataset("Cream/data/imagenet/val")

In [3]:
loader_eval = create_loader(
        dataset_eval,
        input_size = (3, 224, 224),
        batch_size = 4 * 32,
        is_training = True,
        num_workers = 4,
        distributed = False,
        interpolation = 'bicubic',
        crop_pct = 0.875,
        mean = (0.485, 0.456, 0.406),
        std = (0.229, 0.224, 0.225))

C:\Users\LibrarristShalinward\AppData\Local\Programs\Python\Python38\lib\site-packages\paddle\fluid\reader.py:357: UserWarning: DataLoader with multi-process mode is not supported on MacOs and Windows currently. Please use signle-process mode with num_workers = 0 instead
  warnings.warn(


#### - 输出值测试

In [4]:
for _, (input, _) in enumerate(loader_eval):
    break
print(input)

C:\Users\LibrarristShalinward\AppData\Local\Programs\Python\Python38\lib\site-packages\paddle\fluid\dygraph\math_op_patch.py:237: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.int64, but right dtype is paddle.uint8, the right dtype will convert to paddle.int64
  warnings.warn(


Tensor(shape=[128, 3, 224, 224], dtype=float32, place=CPUPlace, stop_gradient=True,
       [[[[ 1.15292394,  1.15292394,  1.15292394, ...,  1.15292394,  1.15292394,  1.18717349],
          [ 1.15292394,  1.15292394,  1.15292394, ...,  1.13579929,  1.13579929,  1.15292394],
          [ 1.15292394,  1.15292394,  1.15292394, ...,  1.13579929,  1.13579929,  1.15292394],
          ...,
          [-1.74115944, -1.74115944, -1.74115944, ...,  0.07406451, -0.13143253, -0.31980482],
          [-1.79253364, -1.82678318, -1.82678318, ..., -0.16568205, -0.50817710, -0.74792367],
          [-1.86103261, -1.87815738, -1.87815738, ..., -0.43967810, -0.85067219, -1.12466824]],

         [[ 0.48529413,  0.48529413,  0.48529413, ...,  0.50280118,  0.50280118,  0.52030814],
          [ 0.48529413,  0.48529413,  0.48529413, ...,  0.50280118,  0.50280118,  0.48529413],
          [ 0.48529413,  0.48529413,  0.48529413, ...,  0.50280118,  0.50280118,  0.48529413],
          ...,
          [-1.93067229, -1.89

## ***loss系列***

<font size = "3">

### pimm.loss.LabelSmoothingCrossEntropy

#### - import测试

In [5]:
from Paddle_Cream.lib.utils.pimm.loss import LabelSmoothingCrossEntropy

#### - 测试方法创建

In [6]:
pd_fun = LabelSmoothingCrossEntropy()

#### - 前向对齐测试

In [7]:
import numpy as np

In [8]:
output = np.random.rand(100,20)
target = np.random.randint(low = 0, high = 20, size = (100))
for i in range(100):
    output[i][target[i]] += 10
    output[i] /= output[i].sum()

In [9]:
import paddle
pd_loss = pd_fun(
    paddle.to_tensor(output, dtype = "float32"), 
    paddle.to_tensor(target))
print(pd_loss)

Tensor(shape=[1], dtype=float32, place=CPUPlace, stop_gradient=True,
       [2.57594633])


In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, smoothing=0.1):
        super(LabelSmoothingCrossEntropy, self).__init__()
        assert smoothing < 1.0
        self.smoothing = smoothing
        self.confidence = 1. - smoothing

    def forward(self, x, target):
        logprobs = F.log_softmax(x, dim=-1)
        nll_loss = -logprobs.gather(dim=-1, index=target.unsqueeze(1))
        nll_loss = nll_loss.squeeze(1)
        smooth_loss = -logprobs.mean(dim=-1)
        loss = self.confidence * nll_loss + self.smoothing * smooth_loss
        return loss.mean()

In [11]:
index = torch.from_numpy(target).type_as(torch.zeros((2,2), dtype = torch.int64))
tc_loss = LabelSmoothingCrossEntropy()(
    torch.from_numpy(output), 
    index)
print(tc_loss)

tensor(2.5759, dtype=torch.float64)


C:\Users\LibrarristShalinward\AppData\Local\Programs\Python\Python38\lib\site-packages\paddle\fluid\dygraph\math_op_patch.py:237: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.int64, but right dtype is paddle.uint8, the right dtype will convert to paddle.int64
  warnings.warn(


## ***models.effcientnet_blocks系列***

#### - import测试

In [1]:
from Paddle_Cream.lib.utils.pimm.models.efficientnet_blocks import ConvBnAct, DepthwiseSeparableConv, drop_path, InvertedResidual,SqueezeExcite

<font size = 3>

### pimm.models.effcientnet_blocks.ConvBnAct

#### - 网络创建

In [2]:
conv_bn_act = ConvBnAct(in_chs = 3, out_chs = 8, kernel_size = 3)

#### - 网络组网与运行测试

In [3]:
import paddle
conv_bn_act_model = paddle.Model(conv_bn_act)
conv_bn_act_model.summary(input_size = (10, 3, 224, 224))

---------------------------------------------------------------------------
 Layer (type)       Input Shape          Output Shape         Param #    
   Conv2D-1     [[128, 3, 224, 224]]  [128, 8, 224, 224]        216      
 BatchNorm2D-1  [[128, 8, 224, 224]]  [128, 8, 224, 224]        32       
    ReLU-1      [[128, 8, 224, 224]]  [128, 8, 224, 224]         0       
Total params: 248
Trainable params: 216
Non-trainable params: 32
---------------------------------------------------------------------------
Input size (MB): 73.50
Forward/backward pass size (MB): 1176.00
Params size (MB): 0.00
Estimated Total Size (MB): 1249.50
---------------------------------------------------------------------------



{'total_params': 248, 'trainable_params': 216}

In [5]:
class ConvBnAct_Model(paddle.nn.Layer):
    def __init__(self) -> None:
        super(ConvBnAct_Model, self).__init__()
        self.core = ConvBnAct(in_chs = 3, out_chs = 8, kernel_size = 3)
    
    @paddle.jit.to_static
    def forward(self, x):
        return self.core(x)

conv_bn_act_model = ConvBnAct_Model()
conv_bn_act_model.eval()

conv_bn_act_model(paddle.randn((10, 3, 224, 224)))

Tensor(shape=[10, 8, 224, 224], dtype=float32, place=CPUPlace, stop_gradient=False,
       [[[[0.69864273, 0.        , 0.        , ..., 0.        , 0.79338437, 0.50822300],
          [0.37105399, 3.37294149, 0.        , ..., 0.44676226, 0.        , 1.18462908],
          [0.        , 0.        , 0.40377831, ..., 0.90494603, 0.        , 0.        ],
          ...,
          [0.        , 0.72260791, 2.41498446, ..., 0.        , 0.        , 0.80267859],
          [0.        , 0.77875960, 3.57500196, ..., 3.55005121, 0.75076687, 0.        ],
          [0.        , 0.99618000, 0.98610067, ..., 1.17962587, 0.82934749, 0.18116000]],

         [[0.15328000, 0.72623283, 0.        , ..., 0.31471029, 0.        , 0.        ],
          [0.24300902, 0.        , 0.        , ..., 0.40041646, 0.        , 0.76972729],
          [0.        , 0.        , 0.        , ..., 0.02424304, 0.47348523, 0.60184836],
          ...,
          [0.        , 1.18240237, 1.18703783, ..., 0.90174913, 0.45622760, 0.     

<font size = 3>

### pimm.models.effcientnet_blocks.DepthwiseSeparableConv

#### - 网络创建

In [4]:
dsc = DepthwiseSeparableConv(in_chs = 8, out_chs = 3)

#### - 网络组网与运行测试

In [5]:
import paddle
dsc_model = paddle.Model(dsc)
dsc_model.summary(input_size = (1, 8, 224, 224))

---------------------------------------------------------------------------
 Layer (type)       Input Shape          Output Shape         Param #    
   Conv2D-1      [[1, 8, 224, 224]]    [1, 8, 224, 224]         72       
 BatchNorm2D-1   [[1, 8, 224, 224]]    [1, 8, 224, 224]         32       
    ReLU-1       [[1, 8, 224, 224]]    [1, 8, 224, 224]          0       
   Conv2D-2      [[1, 8, 224, 224]]    [1, 3, 224, 224]         24       
 BatchNorm2D-2   [[1, 3, 224, 224]]    [1, 3, 224, 224]         12       
  Identity-1     [[1, 3, 224, 224]]    [1, 3, 224, 224]          0       
Total params: 140
Trainable params: 96
Non-trainable params: 44
---------------------------------------------------------------------------
Input size (MB): 1.53
Forward/backward pass size (MB): 12.63
Params size (MB): 0.00
Estimated Total Size (MB): 14.16
---------------------------------------------------------------------------



{'total_params': 140, 'trainable_params': 96}

In [2]:
import paddle
class DSC_Model(paddle.nn.Layer):
    def __init__(self) -> None:
        super(DSC_Model, self).__init__()
        self.core = DepthwiseSeparableConv(in_chs = 8, out_chs = 3)
    
    @paddle.jit.to_static
    def forward(self, x):
        return self.core(x)

dsc_model = DSC_Model()
dsc_model.eval()

dsc_model(paddle.randn((10, 8, 224, 224)))

C:\Users\LibrarristShalinward\AppData\Local\Programs\Python\Python38\lib\site-packages\paddle\fluid\layers\utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  return (isinstance(seq, collections.Sequence) and


Tensor(shape=[10, 3, 224, 224], dtype=float32, place=CPUPlace, stop_gradient=False,
       [[[[-0.14746720, -0.16082746, -0.06460346, ...,  0.10291528,  0.19123688, -0.03071185],
          [ 0.01413354,  0.02793776,  0.08720282, ...,  0.07127486, -0.23570250, -0.06562877],
          [-0.16073152, -0.39072195, -0.22850198, ...,  0.04361411,  0.20340875,  0.08733546],
          ...,
          [ 0.00886333, -0.63019735, -0.18531461, ..., -0.10142030, -0.26101342, -0.13374330],
          [ 0.05334020, -0.05911309,  0.11863746, ..., -0.42433298,  0.02986912, -0.00921106],
          [-0.11048304, -0.26561138,  0.09967162, ..., -0.23496552, -0.39887354, -0.23028849]],

         [[-0.02825833,  0.00372237,  0.07207420, ..., -0.45815131,  0.30864695, -0.00271158],
          [-0.03636653, -0.18300231,  0.03164952, ...,  0.24782285,  0.16825292,  0.28083795],
          [-0.31905568,  0.15328529, -0.11988434, ...,  0.17447144, -0.20456903,  0.10108046],
          ...,
          [ 0.10906643,  0.70

<font size = 3>

### pimm.models.effcientnet_blocks.drop_path

In [4]:
import paddle
test_Tensor = paddle.randn((16, 3, 224, 224))
drop_path(test_Tensor)

Tensor(shape=[16, 3, 224, 224], dtype=float32, place=CPUPlace, stop_gradient=True,
       [[[[ 0.34523138,  0.67044020, -1.35021079, ..., -0.77816898, -0.71464634,  0.70787686],
          [ 0.52097124,  0.06359760, -0.62047458, ...,  0.34788111,  0.17612764,  1.47456205],
          [ 0.72910392,  1.55080426, -0.10572802, ...,  0.67112046,  1.29149270,  0.17798032],
          ...,
          [ 0.30729491, -1.45211172,  0.86003923, ..., -0.07816920, -0.63500702,  0.14781882],
          [-0.12810633, -0.22542620,  1.02429819, ...,  1.07805037,  1.62149918, -0.68539530],
          [ 0.17343253,  0.08267141,  1.21846211, ...,  0.32996413,  0.27107236,  1.63118708]],

         [[-1.69572592,  0.86185658, -0.02888284, ...,  0.88045692,  1.08452797,  0.17446697],
          [-0.80477518,  1.60663891, -0.55515867, ...,  0.22370850,  0.95800823, -0.69524205],
          [ 0.73241055,  0.81839335,  1.06419480, ...,  1.35788119,  0.42697033, -0.68099213],
          ...,
          [ 0.59532386, -1.541

<font size = 3>

### pimm.models.effcientnet_blocks.InvertedResidual

#### - 网络创建

In [5]:
ir = InvertedResidual(in_chs = 8, out_chs = 3)

#### - 网络组网与运行测试

In [7]:
import paddle
ir_model = paddle.Model(ir)
ir_model.summary(input_size = (16, 8, 224, 224))

---------------------------------------------------------------------------
 Layer (type)       Input Shape          Output Shape         Param #    
   Conv2D-4     [[16, 8, 224, 224]]   [16, 8, 224, 224]         64       
 BatchNorm2D-3  [[16, 8, 224, 224]]   [16, 8, 224, 224]         32       
    ReLU-2      [[16, 8, 224, 224]]   [16, 8, 224, 224]          0       
   Conv2D-5     [[16, 8, 224, 224]]   [16, 8, 224, 224]         72       
 BatchNorm2D-4  [[16, 8, 224, 224]]   [16, 8, 224, 224]         32       
    ReLU-3      [[16, 8, 224, 224]]   [16, 8, 224, 224]          0       
   Conv2D-6     [[16, 8, 224, 224]]   [16, 3, 224, 224]         24       
 BatchNorm2D-5  [[16, 3, 224, 224]]   [16, 3, 224, 224]         12       
Total params: 236
Trainable params: 160
Non-trainable params: 76
---------------------------------------------------------------------------
Input size (MB): 24.50
Forward/backward pass size (MB): 330.75
Params size (MB): 0.00
Estimated Total Size (MB): 355.

{'total_params': 236, 'trainable_params': 160}

In [8]:
import paddle
class IR_Model(paddle.nn.Layer):
    def __init__(self) -> None:
        super(IR_Model, self).__init__()
        self.core = InvertedResidual(in_chs = 8, out_chs = 3)
    
    @paddle.jit.to_static
    def forward(self, x):
        return self.core(x)

ir_model = IR_Model()
ir_model.eval()

ir_model(paddle.randn((16, 8, 224, 224)))

Tensor(shape=[16, 3, 224, 224], dtype=float32, place=CPUPlace, stop_gradient=False,
       [[[[-0.62883860, -1.72421718, -0.84774858, ..., -0.30147532, -0.05518850,  0.01213455],
          [-1.19636726, -2.69388390, -2.41328549, ...,  1.00956547, -1.42900872, -1.12430322],
          [-1.46179903, -1.35182071, -2.42473817, ..., -0.36864668, -1.53713739, -0.55700165],
          ...,
          [-1.90067041, -3.94212413, -1.15357733, ..., -1.74033928, -1.59882057, -1.11028016],
          [-1.88147247, -1.02714837, -0.99738479, ..., -1.07954538, -1.74188554, -0.97307825],
          [-0.55896455, -0.68617558, -1.35178649, ..., -1.03353035, -0.51481444, -0.08892007]],

         [[-0.37970936, -0.92431635, -0.55747086, ..., -0.20014901, -0.35214466, -0.70372230],
          [-0.42499259, -1.50434637, -0.93478072, ..., -0.17784496, -0.33792678, -0.41905481],
          [-0.49103761, -0.21434151, -0.95398593, ...,  0.13564308, -0.19860902, -0.47105297],
          ...,
          [-1.37064862, -3.48

<font size = 3>

### pimm.models.effcientnet_blocks.InvertedResidual

#### - 网络创建

In [9]:
se = SqueezeExcite(in_chs = 8)

#### - 网络组网与运行测试

In [10]:
import paddle
se_model = paddle.Model(se)
se_model.summary(input_size = (16, 8, 224, 224))

-------------------------------------------------------------------------------
   Layer (type)         Input Shape          Output Shape         Param #    
AdaptiveAvgPool2D-1 [[16, 8, 224, 224]]     [16, 8, 1, 1]            0       
     Conv2D-12        [[16, 8, 1, 1]]       [16, 2, 1, 1]           18       
      ReLU-6          [[16, 2, 1, 1]]       [16, 2, 1, 1]            0       
     Conv2D-13        [[16, 2, 1, 1]]       [16, 8, 1, 1]           24       
Total params: 42
Trainable params: 42
Non-trainable params: 0
-------------------------------------------------------------------------------
Input size (MB): 24.50
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 24.50
-------------------------------------------------------------------------------



{'total_params': 42, 'trainable_params': 42}

In [11]:
import paddle
class SE_Model(paddle.nn.Layer):
    def __init__(self) -> None:
        super(SE_Model, self).__init__()
        self.core = SqueezeExcite(in_chs = 8)
    
    @paddle.jit.to_static
    def forward(self, x):
        return self.core(x)

se_model = SE_Model()
se_model.eval()

se_model(paddle.randn((16, 8, 224, 224)))

Tensor(shape=[16, 8, 224, 224], dtype=float32, place=CPUPlace, stop_gradient=False,
       [[[[-0.07570963, -0.41265354,  0.39139521, ...,  0.75775397, -0.37889513, -0.86286497],
          [ 0.07384950, -0.34077942, -0.11042850, ..., -0.50394177, -0.59697473,  0.81060326],
          [ 0.50404871, -0.19356698,  0.01846549, ...,  0.80095768, -0.44174397,  0.24296264],
          ...,
          [ 0.46132317,  0.13635097, -0.00051444, ..., -0.53701806,  0.22719491, -0.30654544],
          [ 0.20035037, -0.72983360, -0.28788674, ...,  0.30187964, -0.58554178, -0.62101704],
          [ 0.86163056,  0.85762244,  0.06690049, ..., -0.06390416,  0.33138588, -0.56844294]],

         [[ 0.05276460, -0.23558870,  0.70290613, ..., -0.33514059,  0.04964886,  1.50248003],
          [ 0.62936246, -0.44692108,  1.01719677, ...,  0.46287248, -0.10482739,  0.44738913],
          [-0.59368557,  0.43399271,  0.23767465, ...,  0.70149243, -0.31443700,  0.28431571],
          ...,
          [-0.37197259, -0.98